In [1]:
import numpy as np
import pandas as pd
import os, vcf, sys, itertools, shutil, subprocess, glob, yaml
from Bio import Entrez, Seq, SeqIO
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

H37Rv_regions = pd.read_csv("./references/ref_genome/mycobrowser_h37rv_v4.csv")
isolate_metadata = pd.read_csv("/n/data1/hms/dbmi/farhat/rollingDB/metadata/isolate_metadata.csv")

df_WGS = pd.read_csv("/n/data1/hms/dbmi/farhat/rollingDB/TRUST/Illumina_culture_WGS_summary.csv")

In [2]:
DS_samples = os.listdir("/n/data1/hms/dbmi/farhat/aryan/TRUST/FreeBayesVCFs/AF_replaced_vcfs/DS")

# not sure if we should remove the A and B samples
# DS_samples = [fName.split('_dedup')[0].replace('_', '-').replace('A', '').replace('B', '') for fName in DS_samples]
DS_samples = [fName.split('_')[0].replace('S0', 'T0') for fName in DS_samples]
print(len(DS_samples), len(np.unique(DS_samples)))

DS_samples = np.unique(DS_samples)

206 179


In [3]:
trust_report_fNames = glob.glob("/n/data1/hms/dbmi/farhat/rollingDB/TRUST/WGS_metadata_reports/*")
len(trust_report_fNames)

6

In [4]:
df_trust_WGS_metadata = pd.read_csv("/n/data1/hms/dbmi/farhat/rollingDB/TRUST/clinical_data/combined_patient_WGS_data.csv")

In [193]:
copy_samples = df_trust_WGS_metadata.query("pid in @DS_samples")['Original_ID'].str.replace('_', '-').unique()

def normalize_patient_sample_prefixes(samples_lst):
    '''
    Add S padded with 0s to make the full name S#### so we can match the samples to patients.
    '''
    
    normalized_sample_names = []
    
    for sample in samples_lst:
        
        # I think these are TOTAL samples. Just return the name as is because there's no longitudinal component, suggesting it's not a TRUST sample
        if '-' not in sample:
            normalized_sample_names.append(sample)
        
        else:
            prefix, suffix = sample.split("-", 1)
            if prefix.startswith("S"):
                # already starts with S, just ensure 4 digits after S
                num_part = prefix[1:].zfill(4)  
                new_prefix = f"S{num_part}"
            else:
                # no S, add it and zero-pad to 4 digits
                new_prefix = f"S{prefix.zfill(4)}"
            normalized_sample_names.append(f"{new_prefix}-{suffix}")

    return normalized_sample_names

normalized_sample_names = normalize_patient_sample_prefixes(copy_samples)

df_Aryan_samples = pd.DataFrame({'Name': copy_samples, 'NormName': normalized_sample_names})

In [197]:
df_Aryan_samples.iloc[df_Aryan_samples.index.values[df_Aryan_samples.duplicated(subset='NormName', keep=False)]]

,Name,NormName
5,S0157-01,S0157-01
6,157-01,S0157-01
154,170-01,S0170-01
155,S0170-01,S0170-01


In [7]:
in_dir = "/n/data1/hms/dbmi/farhat/fastq_db/TRUST_Directsputum"
out_dir = "/n/scratch/users/s/sak0914/TRUST_sputum"

In [125]:
FQ_files = glob.glob(f"{in_dir}/*/*.gz")

df_FQ = pd.DataFrame({'FQ': FQ_files})
df_FQ['fName'] = [os.path.basename(fName) for fName in df_FQ['FQ']]
df_FQ['dirName'] = [os.path.basename(os.path.dirname(fName)) for fName in df_FQ['FQ']]
len(df_FQ)

542

In [199]:
df_FQ['Sample'] = [os.path.basename(fName).split('_S')[0].split('_R')[0].replace('_', '-') for fName in df_FQ['FQ']]

# normalize names
df_FQ['Sample'] = normalize_patient_sample_prefixes(df_FQ['Sample'])

df_FQ['ReadNum'] = [os.path.basename(fName).split('_R')[1].split('_')[0].split('.')[0] for fName in df_FQ['FQ']]
print(df_FQ.ReadNum.unique())

df_FQ['ReadNum'] = df_FQ['ReadNum'].astype(int)
df_FQ[['Original_ID', 'SampleNum']] = df_FQ['Sample'].str.split('-', expand=True)

['1' '2']


In [200]:
df_FQ_to_run = df_FQ.drop_duplicates(subset=['Sample', 'ReadNum'], keep=False)
len(df_FQ_to_run)

388

In [201]:
df_FQ.query("fName.str.contains('066')")

,FQ,fName,dirName,Sample,ReadNum,Original_ID,SampleNum
24,/n/data1/hms/dbmi/farhat/fastq_db/TRUST_Direct...,S0066_01_R1.fastq.gz,20220509_TwistV1,S0066-01,1,S0066,01
25,/n/data1/hms/dbmi/farhat/fastq_db/TRUST_Direct...,S0066_01_R2.fastq.gz,20220509_TwistV1,S0066-01,2,S0066,01


In [203]:
df_FQ_to_run.query("Sample in @normalized_sample_names").Sample.nunique()

117

In [204]:
df_FQ_to_copy = df_FQ_to_run.query("Sample in @normalized_sample_names")
print(len(df_FQ_to_copy))

df_FQ_to_copy.query("FQ.str.contains('066')")

234


,FQ,fName,dirName,Sample,ReadNum,Original_ID,SampleNum
24,/n/data1/hms/dbmi/farhat/fastq_db/TRUST_Direct...,S0066_01_R1.fastq.gz,20220509_TwistV1,S0066-01,1,S0066,01
25,/n/data1/hms/dbmi/farhat/fastq_db/TRUST_Direct...,S0066_01_R2.fastq.gz,20220509_TwistV1,S0066-01,2,S0066,01


In [205]:
missing_samples = list(set(df_FQ_to_run.Sample) - set(df_FQ_to_run.query("Sample in @normalized_sample_names").Sample))
len(missing_samples)

77

In [206]:
df_FQ_to_copy.dirName.unique()

array(['20220509_TwistV1', '20240819_TwistV2'], dtype=object)

In [207]:
# for i, row in df_FQ_to_copy.iterrows():

#     os.makedirs(f"{out_dir}/{row['Sample']}", exist_ok=True)
    
#     shutil.copy(row['FQ'], f"{out_dir}/{row['Sample']}/{row['Sample']}_R{row['ReadNum']}.fastq.gz")

In [212]:
df_samples = df_FQ_to_copy[['Sample', 'Sample']].drop_duplicates()
df_samples['FQ'] = 0

df_samples.to_csv("~/longitudinal_changes/Mtb_low_frequency_variants/TRUST_DS_isolates.tsv", sep='\t', header=None, index=False)

In [13]:
df = pd.DataFrame({'A': 'S0326-01',
                   'B': 'S0326-01',
                   'C': 0
                  }, index=[0])

In [16]:
df.to_csv("/home/sak0914/longitudinal_changes/Mtb_low_frequency_variants/TRUST_DS_isolates.tsv", sep='\t', header=None, index=False)

In [36]:
df_kraken = pd.read_csv("/n/data1/hms/dbmi/farhat/Sanjana/TRUST_direct_sputum/S0326-01/S0326-01/kraken/kraken_report_standard_DB.txt", sep='\t', header=None,
                        names=['Percent', 'CumulCount', 'Count', 'TaxLevel', 'Taxid', 'Name']
                       )

df_kraken['Name'] = df_kraken['Name'].str.strip()

In [40]:
df_kraken.query("Percent > 0 & TaxLevel in ['G']").sort_values('Percent', ascending=False)#.Name.values

,Percent,CumulCount,Count,TaxLevel,Taxid,Name
31,53.03,379055,0,G,9605,Homo
451,16.64,118902,98625,G,1763,Mycobacterium
39,14.54,103925,63260,G,1301,Streptococcus
212,9.83,70278,37791,G,1378,Gemella
807,1.57,11236,184,G,32207,Rothia
373,0.33,2375,1513,G,29465,Veillonella
813,0.12,848,359,G,1269,Micrococcus
683,0.09,644,104,G,1716,Corynebacterium
218,0.07,517,28,G,1386,Bacillus
817,0.05,328,121,G,57493,Kocuria


In [8]:
not_found_DS_samples = list(set(DS_samples) - set(df_trust_WGS_metadata['pid']))
len(not_found_DS_samples)

8

In [9]:
not_found_DS_samples

['T0454', 'T0438', 'T0281', 'T0107Cntrl', 'T0476', 'K064773', 'T0348', 'H37']

In [103]:
def combine_TRUST_patient_samples(df_trust_patient_data, WGS_metadata):

    print(f"{len(df_trust_patient_data.patient_num.unique())} patients in phenotypes dataframe")
    
    # each sample ID is in the format like 260-04, S0299-01, or S0271-01A. So use re to remove all alpha characters and keep only numbers.
    # first number is the patient number, second is the sample number
    # merge the patient number extracted from each sample with the patient numbers in df_trust_patient_data
    for i, row in WGS_metadata.iterrows():
        try:
            patient_num = int(re.findall(r'\d+', row["Original_ID"].split("-")[0])[0])
        # weird names like T-7, T-5, and P-T- 1
        except:
            patient_num = row["Original_ID"]

        try:
            sample_week = int(re.findall(r'\d+', row["Original_ID"].split("-")[1])[0])
        # weird names like T-7, T-5, and P-T- 1
        except:
            sample_week = row['Original_ID']
            
        WGS_metadata.loc[i, ['patient_num', 'sample_week']] = [patient_num, sample_week]
            
    # WGS_metadata["patient_num"] = [int(re.findall(r'\d+', val.split("-")[0])[0]) for val in WGS_metadata["Original_ID"].values]
    # WGS_metadata["sample_week"] = [int(re.findall(r'\d+', val.split("-")[1])[0]) for val in WGS_metadata["Original_ID"].values]

    # keep only samples for the patients in df_trust_patient_data
    WGS_metadata = WGS_metadata.merge(df_trust_patient_data, on="patient_num", how="inner")
    print(f"Found {WGS_metadata.patient_num.nunique()}/{df_trust_patient_data.patient_num.nunique()} patients in the WGS metadata files")
    
    # rename some columns that have spaces in them
    WGS_metadata = WGS_metadata.rename(columns={"Cov Any Mean": "Cov_Any_Mean",
                                                "Cov Unam Perc": "Cov_Unam_Perc",
                                                "Perc. Reads Mapped": "Perc_Reads_Mapped",
                                                "phylogenetic classification (Coll et al., 2014)": "Coll2014_Annotated"
                                       })

    return WGS_metadata


In [106]:
df

,SampleID,Original_ID,shipment,status,comment,SampleLib,"phylogenetic classification (Coll et al., 2014)",quality,Cov Any Mean,Cov Unam Perc,Perc. Reads Mapped
0,MFS-179,260-04,1st,failed,heavily contaminated,MFS-179_lib48359,2.2.1.1 Beijing,4,6.65,0.00,0.0916
1,MFS-78,157-01,1st,failed,heavily contaminated,MFS-78_lib47065,4.6.2.1 Euro-American Ancestral 2,3,147.41,0.70,0.4469
2,MFS-183,264-07,1st,failed,heavily contaminated,MFS-183_lib48363,2.2.1 Beijing,1,186.20,0.97,0.6105
3,MFS-141,231-05,1st,finished,-,MFS-141_lib47320,4.3.2.1 LAM,1,268.19,1.00,0.9833
4,MFS-188,268-09,1st,finished,contaminated,MFS-188_lib48368,2.2.1 Beijing Asian/Africa 2,1,282.35,0.99,0.8306
...,...,...,...,...,...,...,...,...,...,...,...
262,MFS-284,S0295-05,2nd,finished,-,MFS-284_lib52225,2.2.1.1 Beijing Pacific RD150,1,234.63,0.99,0.9873
263,MFS-285,S0297-01,2nd,finished,-,MFS-285_lib52226,4.3.4.1 LAM,1,233.53,0.99,0.9900
264,MFS-302,S0286-08,2nd,finished,-,MFS-302_lib52243,2.2.1.1 Beijing Pacific RD150,1,215.07,0.99,0.9916
265,MFS-259,S0299-01,2nd,finished,-,MFS-259_lib51729,2.2.1 Beijing Asian/Africa 2,1,184.20,0.99,0.9866


In [107]:
patient_data_fName = "/n/data1/hms/dbmi/farhat/rollingDB/TRUST/clinical_data/TRUST_DATA_2025-05-12_1129.cleaned.wide.csv"
WGS_report_directory = "/n/data1/hms/dbmi/farhat/rollingDB/TRUST/WGS_metadata_reports"
WGS_data_fName = "/n/data1/hms/dbmi/farhat/rollingDB/TRUST/Illumina_culture_WGS_summary.csv"

df_trust_patient_data = pd.read_csv(patient_data_fName, low_memory=False)

# get the patient number to match WGS IDs and pids
df_trust_patient_data["patient_num"] = [int(patient_id.replace("T0", "")) for patient_id in df_trust_patient_data["pid"].values]

# get all Excel files from this directory
trust_report_fNames = glob.glob(f"{WGS_report_directory}/*.xlsx")
print(f"{len(trust_report_fNames)} WGS metadata Excel files")

df_trust_WGS_metadata = []

# sort chronologically because below, we preferentially keep the later one
for fName in np.sort(trust_report_fNames)[::-1]:

    # read in single Excel file
    df = pd.read_excel(fName, sheet_name=None)

    # remove spaces from column name to make querying easier. Also there could be NaN rows if there are additional empty rows in the Excel sheet, so drop them
    df = df['summary'].rename(columns={'original ID': 'Original_ID'}).dropna(axis=0, how='all')

    # append to list for concatenation
    df_trust_WGS_metadata.append(df)

# the Excel files above are running totals, so the most recent file has data that is also in the older files. So drop duplicates, keeping the most recent (last) one
df_trust_WGS_metadata = pd.concat(df_trust_WGS_metadata).drop_duplicates('SampleID', keep='last')#.query("status!='failed'")

# combine patient and sample IDs (pid = patient, Original_ID and SampleID = WGS)
df_trust_combined = combine_TRUST_patient_samples(df_trust_patient_data, df_trust_WGS_metadata.reset_index(drop=True))

# combine with additional WGS QC data. Drop samples that filed QC (no VCF, so they have nothing in the Lineage column)
df_geno = pd.read_csv(WGS_data_fName).dropna(subset='Lineage')

# df_trust_combined = df_trust_combined.merge(df_geno[['SampleID', 'F2', 'Coll2014', 'Freschi2020', 'Lineage']], on='SampleID', how='left').reset_index(drop=True)

# # Remove columns that are NaN everywhere to clean the dataframe
# df_trust_combined = df_trust_combined.dropna(how='all', axis=1)

# # get the sample week
# df_trust_combined['Sampling_Week'] = df_trust_combined['Original_ID'].str.split('-').str[1]

# # replace month 5 with 20 for weeks
# df_trust_combined['Sampling_Week'] = df_trust_combined['Sampling_Week'].replace('01A', '01').replace('m5', '20').replace('M5', '20')

6 WGS metadata Excel files
458 patients in phenotypes dataframe
Found 0/458 patients in the WGS metadata files


In [111]:
trust_report_fNames

['/n/data1/hms/dbmi/farhat/rollingDB/TRUST/WGS_metadata_reports/2023-01-24_report_Farhat.v09.xlsx',
 '/n/data1/hms/dbmi/farhat/rollingDB/TRUST/WGS_metadata_reports/2023-09-01_report_Farhat.v09.xlsx',
 '/n/data1/hms/dbmi/farhat/rollingDB/TRUST/WGS_metadata_reports/2024-05-27_report.v10.xlsx',
 '/n/data1/hms/dbmi/farhat/rollingDB/TRUST/WGS_metadata_reports/2024-06-25_report.v10.xlsx',
 '/n/data1/hms/dbmi/farhat/rollingDB/TRUST/WGS_metadata_reports/2024-11-13_report.v10.xlsx',
 '/n/data1/hms/dbmi/farhat/rollingDB/TRUST/WGS_metadata_reports/2025-07-25_report.v10.xlsx']

In [128]:
# read in single Excel file
# df = pd.read_excel('/n/data1/hms/dbmi/farhat/rollingDB/TRUST/WGS_metadata_reports/2025-07-25_report.v10.xlsx', sheet_name=None)
df = pd.read_excel('/n/data1/hms/dbmi/farhat/fastq_db/TRUST_Illumina/250730_IlluminaWGS_lastbatch/2025-07-25_report.v10.xlsx', sheet_name=None)

# remove spaces from column name to make querying easier. Also there could be NaN rows if there are additional empty rows in the Excel sheet, so drop them
df = df['summary'].rename(columns={'original ID': 'Original_ID'}).dropna(axis=0, how='all')

df['Original_ID'] = df['Original_ID'].astype(str)

In [129]:
df.query("SampleID=='MFS-918'")

,SampleID,SampleLib,Original_ID,Pacbio,Comment,quality,Cov Any Mean,Cov Unam Perc,Perc. Reads Mapped,Excluded,repeat_pcrfree_1,repeat_pcrfree_2,transfered_pcrfree,transfered_pacbio,Repeat_1_factor,repat_1_loading
18,MFS-918,MFS-918_lib84065,TS373,NaN,NaN,1,283.69,1,0.9846,0,NaN,NaN,0,0,NaN,NaN


In [132]:
df.query("Original_ID.str.contains('TS')")

,SampleID,SampleLib,Original_ID,Pacbio,Comment,quality,Cov Any Mean,Cov Unam Perc,Perc. Reads Mapped,Excluded,repeat_pcrfree_1,repeat_pcrfree_2,transfered_pcrfree,transfered_pacbio,Repeat_1_factor,repat_1_loading
18,MFS-918,MFS-918_lib84065,TS373,NaN,NaN,1,283.69,1,0.9846,0,NaN,NaN,0,0,NaN,NaN
22,MFS-919,MFS-919_lib84066,TS374,NaN,NaN,1,273.82,0.99,0.9812,0,NaN,NaN,0,0,NaN,NaN
25,MFS-911,MFS-911_lib84058,TS95,NaN,NaN,1,263.04,0.99,0.9821,0,NaN,NaN,0,0,NaN,NaN
29,MFS-923,MFS-923_lib84070,TS586,NaN,NaN,1,313.35,0.99,0.9851,0,1,NaN,0,0,-0.106431,-0.319292
33,MFS-921,MFS-921_lib84068,TS465,NaN,NaN,1,291.56,0.99,0.9852,0,1,NaN,0,0,-0.039649,-0.118946
37,MFS-912,MFS-912_lib84059,TS128,NaN,NaN,1,280.54,0.99,0.9834,0,1,NaN,0,0,-0.001925,-0.005775
38,MFS-914,MFS-914_lib84061,TS168,NaN,NaN,1,269.2,0.99,0.9858,0,1,NaN,0,0,0.040119,0.120357
40,MFS-913,MFS-913_lib84060,TS147,NaN,NaN,1,329.17,0.99,0.9844,0,1,NaN,0,0,-0.149376,-0.448127
42,MFS-910,MFS-910_lib84057,TS12,NaN,NaN,1,312.2,1,0.9842,0,1,NaN,0,0,-0.103139,-0.309417
43,MFS-915,MFS-915_lib84062,TS183,NaN,NaN,1,314.4,1,0.9841,0,1,NaN,0,0,-0.109415,-0.328244


In [130]:
df.query("Original_ID.str.contains('TS')")

,SampleID,SampleLib,Original_ID,Pacbio,Comment,quality,Cov Any Mean,Cov Unam Perc,Perc. Reads Mapped,Excluded,repeat_pcrfree_1,repeat_pcrfree_2,transfered_pcrfree,transfered_pacbio,Repeat_1_factor,repat_1_loading
18,MFS-918,MFS-918_lib84065,TS373,NaN,NaN,1,283.69,1,0.9846,0,NaN,NaN,0,0,NaN,NaN
22,MFS-919,MFS-919_lib84066,TS374,NaN,NaN,1,273.82,0.99,0.9812,0,NaN,NaN,0,0,NaN,NaN
25,MFS-911,MFS-911_lib84058,TS95,NaN,NaN,1,263.04,0.99,0.9821,0,NaN,NaN,0,0,NaN,NaN
29,MFS-923,MFS-923_lib84070,TS586,NaN,NaN,1,313.35,0.99,0.9851,0,1,NaN,0,0,-0.106431,-0.319292
33,MFS-921,MFS-921_lib84068,TS465,NaN,NaN,1,291.56,0.99,0.9852,0,1,NaN,0,0,-0.039649,-0.118946
37,MFS-912,MFS-912_lib84059,TS128,NaN,NaN,1,280.54,0.99,0.9834,0,1,NaN,0,0,-0.001925,-0.005775
38,MFS-914,MFS-914_lib84061,TS168,NaN,NaN,1,269.2,0.99,0.9858,0,1,NaN,0,0,0.040119,0.120357
40,MFS-913,MFS-913_lib84060,TS147,NaN,NaN,1,329.17,0.99,0.9844,0,1,NaN,0,0,-0.149376,-0.448127
42,MFS-910,MFS-910_lib84057,TS12,NaN,NaN,1,312.2,1,0.9842,0,1,NaN,0,0,-0.103139,-0.309417
43,MFS-915,MFS-915_lib84062,TS183,NaN,NaN,1,314.4,1,0.9841,0,1,NaN,0,0,-0.109415,-0.328244


In [22]:
df_Anna = pd.read_csv("~/Mtb_Megapipe/isolates_for_Anna.tsv", sep='\t', header=None)

In [23]:
for i, row in df_Anna.iterrows():
    
    if os.path.isfile(f"/n/data1/hms/dbmi/farhat/rollingDB/genomic_data/{row[0]}/pilon/{row[0]}_variants.vcf"):
        df_Anna.loc[i, 'Done'] = 1

In [20]:
df_Anna.Done.value_counts(dropna=False)

Done
NaN    13985
1.0     8470
Name: count, dtype: int64

In [24]:
df_Anna.Done.value_counts(dropna=False)

Done
NaN    15367
1.0     7088
Name: count, dtype: int64

In [25]:
PZA_isolates = pd.read_csv("~/Mtb_Megapipe/PZA_isolates_5.tsv", sep='\t', header=None)

In [27]:
PZA_isolates.loc[PZA_isolates[1]=='SRR6831820']

,0,1,2
240,SAMN08708749,SRR6831820,1


In [7]:
df_kraken = pd.read_csv("~/TRUST_lowAF_keep_kraken_proportion.csv")

In [16]:
df_kraken.sort_values("Keep_Proportion").merge(df_WGS[['SampleID', 'Kraken_Unclassified_Percent']]).head(20)

,SampleID,Total_Reads,Keep_Reads,Keep_Proportion,Kraken_Unclassified_Percent
0,MFS-605,6824968,5509747,0.807293,18.03
1,MFS-188,4621344,3825199,0.827724,17.48
2,MFS-366,6097976,5262188,0.862940,13.22
3,MFS-778,5448727,4736817,0.869344,12.82
4,MFS-409,4951776,4478437,0.904410,10.60
5,MFS-138,5083364,4683315,0.921302,7.09
6,MFS-397,5586986,5283277,0.945640,4.99
7,MFS-158,4817209,4591883,0.953225,3.87
8,MFS-776,4899988,4735179,0.966365,2.69
9,MFS-157,3713859,3596537,0.968410,2.27


In [63]:
isolate_metadata.query("DB_OF_ORIGIN=='CASS'").head()

,ROLLINGDB_ID,BIOSAMPLE_ACCESSION,SAMPLE,RUN,DB_OF_ORIGIN,ISOLATION_DATE,ISOLATION_COUNTRY,ISOLATION_REGION,SPUTUM/CULTURE,UNPAIRED/PAIRED,Platform,Model,SPECIES,TaxID,ReleaseDate,LoadDate,Combined_Runs,Num_Runs,OtherNames
698,C001,NaN,NaN,NaN,CASS,NaN,NaN,NaN,1,1,NaN,NaN,Mycobacterium tuberculosis,NaN,NaN,NaN,NaN,NaN,NaN
699,C002,NaN,NaN,NaN,CASS,NaN,NaN,NaN,1,1,NaN,NaN,Mycobacterium tuberculosis,NaN,NaN,NaN,NaN,NaN,NaN
700,C003,NaN,NaN,NaN,CASS,NaN,NaN,NaN,1,1,NaN,NaN,Mycobacterium tuberculosis,NaN,NaN,NaN,NaN,NaN,NaN
701,C004,NaN,NaN,NaN,CASS,NaN,NaN,NaN,1,1,NaN,NaN,Mycobacterium tuberculosis,NaN,NaN,NaN,NaN,NaN,NaN
702,C005,NaN,NaN,NaN,CASS,NaN,NaN,NaN,1,1,NaN,NaN,Mycobacterium tuberculosis,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
cass_fastqs = glob.glob(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/C*")
len(cass_fastqs)

313

In [5]:
cass_fastqs_isolates = [os.path.basename(fName) for fName in cass_fastqs]
problem_cass_fastqs = [os.path.basename(fName) for fName in glob.glob(f"/n/data1/hms/dbmi/farhat/rollingDB/problem_fastqs/C*")]

len(cass_fastqs_isolates), len(problem_cass_fastqs)

(313, 54)

In [6]:
cass_mapping_table.loc[cass_mapping_table['Sample Name'].isin(problem_cass_fastqs)][['BioSample', 'Run', 'Sample Name', 'geo_loc_name_country', 'geo_loc_name', 'Instrument', 'Platform', 'ReleaseDate', 'Collection_Date']].shape

(54, 9)

In [7]:
check_fastas = cass_mapping_table.loc[cass_mapping_table['Sample Name'].isin(cass_fastqs_isolates)][['BioSample', 'Run', 'Sample Name', 'geo_loc_name_country', 'geo_loc_name', 'Instrument', 'Platform', 'ReleaseDate', 'Collection_Date']].reset_index(drop=True)

print(check_fastas.shape)

check_fastas.head()

(307, 9)


,BioSample,Run,Sample Name,geo_loc_name_country,geo_loc_name,Instrument,Platform,ReleaseDate,Collection_Date
0,SAMN13813810,SRR10868968,C429,South Africa,South Africa,Illumina HiSeq 2500,ILLUMINA,2020-04-28T00:00:00Z,2019
1,SAMN13813725,SRR10868973,C094,South Africa,South Africa,Illumina HiSeq 2500,ILLUMINA,2020-04-28T00:00:00Z,2019
2,SAMN13813804,SRR10868975,C412,South Africa,South Africa,Illumina HiSeq 2500,ILLUMINA,2020-04-28T00:00:00Z,2019
3,SAMN13813803,SRR10868976,C408,South Africa,South Africa,Illumina HiSeq 2500,ILLUMINA,2020-04-28T00:00:00Z,2019
4,SAMN13813802,SRR10868977,C407,South Africa,South Africa,Illumina HiSeq 2500,ILLUMINA,2020-04-28T00:00:00Z,2019


In [8]:
def count_lines(file_path):
    # Use subprocess to call wc -l to count the number of lines in a file
    process = subprocess.Popen(['wc', '-l', file_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    if process.returncode != 0:
        print(f"Error counting lines in {file_path}: {stderr.decode().strip()}")
        return None

    # Extract and return the line count from the output
    line_count = int(stdout.decode().strip().split()[0])
    return line_count

In [13]:
good_files = []
bad_files = []

for i, row in check_fastas.iterrows():

    internal_id = row['Sample Name']
    run_id = row['Run']

    # if not os.path.isfile(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_1.fastq") or not os.path.isfile(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_2.fastq"):

    #     subprocess.run(f"gunzip -c /n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_R1.fastq.gz > /n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_1.fastq", shell=True)

    #     subprocess.run(f"gunzip -c /n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_R2.fastq.gz > /n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_2.fastq", shell=True)

    # if i % 25 == 0:
    #     print(f"Finished unzipping sample {i}")
        
    F1_LC_internal = count_lines(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_1.fastq")
    F2_LC_internal = count_lines(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_2.fastq")

    if os.path.isfile(f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_1.fastq") and os.path.isfile(f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_2.fastq"):
        
        F1_LC_public = count_lines(f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_1.fastq")
        F2_LC_public = count_lines(f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_2.fastq")

        if F1_LC_internal == F2_LC_internal == F1_LC_public == F2_LC_public:
            good_files.append(run_id)
        else:
            bad_files.append(run_id)

    if i % 50 == 0:
        print(i)

len(good_files), len(bad_files)

0
50
100
150
200
250
300


(304, 3)

In [17]:
bad_files

['SRR10869030', 'SRR10869302', 'SRR10869310']

In [18]:
def remove_headers_FASTQ(fName):
    subprocess.run(f"awk 'NR % 2 == 0' {fName} > {fName.replace('_1', '_noID_1').replace('_2', '_noID_2')}", shell=True)

In [19]:
def compare_files(file1, file2):
    result = subprocess.run(['diff', file1, file2], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode == 0:
        # print("Files are the same.")
        return 0
    else:
        # print("Files are different.")
        return 1

In [20]:
compare_files('/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/C001/C001_noID_1.fastq', '/n/scratch/users/s/sak0914/smk_output/C001/SRR10869296/SRR10869296_noID_1.fastq')

0

In [23]:
compare_files('/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/C429/C429_noID_1.fastq', '/n/scratch/users/s/sak0914/smk_output/C429/SRR10868968/SRR10868968_noID_1.fastq')

1

In [22]:
df_compare_FASTQ

,ID,FQ1,FQ2
0,C429,1,1


In [25]:
df_compare_FASTQ = pd.DataFrame(columns=['ID', 'FQ1', 'FQ2'])

for i, row in check_fastas.iterrows():

    internal_id = row['Sample Name']
    run_id = row['Run']

    if not os.path.isfile(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_noID_1.fastq"):
        remove_headers_FASTQ(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_1.fastq")

    if not os.path.isfile(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_noID_2.fastq"):
        remove_headers_FASTQ(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_2.fastq")
        
    if not os.path.isfile(f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_noID_1.fastq"):
        remove_headers_FASTQ(f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_1.fastq")

    if not os.path.isfile(f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_noID_2.fastq"):
        remove_headers_FASTQ(f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_2.fastq")

    cmp_1 = compare_files(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_noID_1.fastq", f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_noID_1.fastq")

    cmp_2 = compare_files(f"/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/{internal_id}/{internal_id}_noID_2.fastq", f"/n/scratch/users/s/sak0914/smk_output/{internal_id}/{run_id}/{run_id}_noID_2.fastq")
    
    df_compare_FASTQ.loc[i, :] = [internal_id, cmp_1, cmp_2]
    
    if i % 50 == 0:
        print(i)

0



KeyboardInterrupt



In [27]:
# for fName in glob.glob("/n/data1/hms/dbmi/farhat/rollingDB/fastq_db/*/*_noID*"):
#     os.remove(fName)

In [29]:
# for fName in glob.glob("/n/scratch/users/s/sak0914/smk_output/*/*/*_noID*"):
#     os.remove(fName)

In [16]:
cass_mapping_table_for_megapipe = cass_mapping_table[['BioSample', 'Run']]
cass_mapping_table_for_megapipe['FASTQ'] = 1
cass_mapping_table_for_megapipe

,BioSample,Run,FASTQ
0,SAMN13813815,SRR10868963,1
1,SAMN13813814,SRR10868964,1
2,SAMN13813813,SRR10868965,1
3,SAMN13813812,SRR10868966,1
4,SAMN13813811,SRR10868967,1
...,...,...,...
356,SAMN13813817,SRR10869319,1
357,SAMN13813816,SRR10869320,1
358,SAMN13813726,SRR10869321,1
359,SAMN13813717,SRR10869322,1


In [2]:
def compare_fasta_sequences(file1, file2):
    # Read sequences from both files into sets
    sequences1 = {str(record.seq) for record in SeqIO.parse(file1, "fasta")}
    sequences2 = {str(record.seq) for record in SeqIO.parse(file2, "fasta")}
    
    # Compare the sets of sequences
    if sequences1 == sequences2:
        return 0
        
    return 1

In [3]:
diff_fastas = {}
alns_lst = []

for fName in glob.glob("kraken_db/PGAP/*"):
    sample_ID = os.path.basename(fName).split('.')[0]

    fName_2 = fName.replace('PGAP', 'Bakta')
    assert os.path.isfile(fName_2)

    return_val = compare_fasta_sequences(fName, fName_2)

    if return_val == 1:
        print(fName, fName_2)
    
    diff_fastas[sample_ID] = return_val

kraken_db/PGAP/S0085-01.HybridAsm.PGAP.Genome.fasta kraken_db/Bakta/S0085-01.HybridAsm.Bakta.Genome.fasta
kraken_db/PGAP/S0107-01.HybridAsm.PGAP.Genome.fasta kraken_db/Bakta/S0107-01.HybridAsm.Bakta.Genome.fasta


In [4]:
len(os.listdir("kraken_db/PGAP")), len(os.listdir("kraken_db/Bakta"))

(151, 151)

In [5]:
diff_fastas_df = pd.DataFrame(diff_fastas, index=[0]).T.reset_index()
diff_fastas_df.columns = ['sample', 'seq_diff']
diff_fastas_df.shape

(151, 2)

In [6]:
diff_fastas_df.query("seq_diff==1")

,sample,seq_diff
85,S0085-01,1
88,S0107-01,1


In [7]:
sample_id = 'S0085-01'
seq_PGAP = [(seq.id, seq.seq) for seq in SeqIO.parse(f"kraken_db/PGAP/{sample_id}.HybridAsm.PGAP.Genome.fasta", "fasta")]
seq_Bakta = [(seq.id, seq.seq) for seq in SeqIO.parse(f"kraken_db/Bakta/{sample_id}.HybridAsm.Bakta.Genome.fasta", "fasta")]

In [8]:
len(seq_PGAP[0][1]), len(seq_Bakta[0][1])

(4417316, 4417384)

In [18]:
S0085_check = "/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/TRUST_PB_Set1/S0085-01.LR.Asm.fasta"
S0085_check_fastq = [(seq.id, seq.seq) for seq in SeqIO.parse(S0085_check, "fasta")]

S0107_check = "/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/TRUST_PB_Set1/S0107-01.LR.Asm.fasta"
S0107_check_fastq = [(seq.id, seq.seq) for seq in SeqIO.parse(S0107_check, "fasta")]

In [19]:
len(S0085_check_fastq[0][1]), len(S0107_check_fastq[0][1])

(4417384, 4416462)

In [11]:
sample_id = 'S0107-01'
seq_PGAP = [(seq.id, seq.seq) for seq in SeqIO.parse(f"kraken_db/PGAP/{sample_id}.HybridAsm.PGAP.Genome.fasta", "fasta")]
seq_Bakta = [(seq.id, seq.seq) for seq in SeqIO.parse(f"kraken_db/Bakta/{sample_id}.HybridAsm.Bakta.Genome.fasta", "fasta")]

In [12]:
len(seq_PGAP[0][1]), len(seq_Bakta[0][1])

(4416405, 4416462)

# Create BED file to exclude sites with low EBR (refined low confidence) or are within 50 bp of a PE/PPE gene

In [3]:
# exclude_fName_Marin = 'RLC_Regions.Plus.LowPmapK50E4.H37Rv'

# RLC_regions = pd.read_csv(f"references/ref_genome/{exclude_fName_Marin}.bed", sep='\t', header=None)
# RLC_regions.columns = ['CHR', 'BEG', 'END']
# RLC_regions['CHR'] = 'Chromosome'
# len(RLC_regions)

In [135]:
# exclude PE/PPE/PGRS genes, sites with EBR < 0.99, and homoplasic sites from Vargas et al., PNAS 2023 
EBR_fName_Marin = 'EBR_Marin.bedgraph'

df_EBR = pd.read_csv(f"references/ref_genome/{EBR_fName_Marin}", sep='\t', header=None)
df_EBR.columns = ['CHR', 'BEG', 'END', 'EBR']
df_EBR['CHR'] = 'Chromosome'

exclude_genes = H37Rv_regions.query("Name.str.startswith('PE') | Name.str.startswith('PPE') | Name.str.contains('PGRS')").Name.values

PE_PPE_BED_file_df = pd.DataFrame(columns=['CHR', 'BEG', 'END'])

for i, gene in enumerate(exclude_genes):

    # these are 1-indexed inclusive
    gene_start, gene_end = H37Rv_regions.set_index('Name').loc[gene, ['Start', 'Stop']].values

    # BED file is 0-indexed half open interval
    # also extend the range by 50 base pairs on either side
    gene_start = gene_start - 1 - 50
    gene_end = gene_end + 50

    PE_PPE_BED_file_df.loc[i, :] = ['Chromosome', gene_start, gene_end]

homoplasy = pd.read_excel("./references/phylogeny/Vargas_PNAS_2023_homoplasy.xlsx", sheet_name=None)

if len(homoplasy.keys()) == 1:
    homoplasy = homoplasy['Sheet1']

homoplasic_sites_BED_file = pd.DataFrame(columns=['CHR', 'BEG', 'END'])

for i, row in homoplasy.iterrows():
    homoplasic_sites_BED_file.loc[i, :] = ['Chromosome', row['H37Rv Position']-1, row['H37Rv Position']]

BED_file_df = pd.concat([PE_PPE_BED_file_df, df_EBR.query("EBR < 0.99")[['CHR', 'BEG', 'END']], homoplasic_sites_BED_file]).sort_values(["BEG", "END"], ascending=True)
print(len(BED_file_df))

BED_file_df.sort_values(['BEG', 'END']).to_csv("./references/phylogeny/exclude_regions_pre_merge.bed", sep='\t', header=None, index=False)

19924


In [115]:
bedtools merge -i /home/sak0914/Mtb_Megapipe/references/phylogeny/exclude_regions_pre_merge.bed > /home/sak0914/Mtb_Megapipe/references/phylogeny/exclude_regions.bed

19924


In [ ]:
gunzip -c SAMEA3392614_full.vcf.gz | bcftools filter -e "SVTYPE == 'INS' | SVTYPE == 'DEL' | IC > 0 | DC > 0" | bedtools subtract -a '-' -b '/home/sak0914/Mtb_Megapipe/references/phylogeny/exclude_regions_combined.bed' | SnpSift extractFields '-' POS REF ALT FILTER QUAL AF DP BQ MQ -e "" >  /n/scratch/users/s/sak0914/smk_output/SAMEA3392614/lineage/SNP_sites.tsv

In [144]:
pd.read_csv("/home/sak0914/Mtb_Megapipe/references/phylogeny/exclude_regions.bed", sep='\t', header=None)

,0,1,2
0,Chromosome,0,33
1,Chromosome,174,185
2,Chromosome,188,211
3,Chromosome,212,244
4,Chromosome,274,275
...,...,...,...
3770,Chromosome,4408093,4408094
3771,Chromosome,4408919,4408920
3772,Chromosome,4411370,4411400
3773,Chromosome,4411469,4411471


In [152]:
def create_SNP_matrix_from_VCF(isolate, qual_thresh=10, min_read_depth=5, BQ_thresh=20, MQ_thresh=30):

    # for encoding ambiguous alleles as possibly both nucleotides
    iupac_two_nuc_codes = {
                            'AG': 'R',  # Purine (A or G)
                            'CT': 'Y',  # Pyrimidine (C or T)
                            'CG': 'S',  # Strong (C or G)
                            'AT': 'W',  # Weak (A or T)
                            'GT': 'K',  # Keto (G or T)
                            'AC': 'M',  # Amino (A or C),
                            'AA': 'A',  # the following are so that you don't get a key error, but they don't actually need to be merged
                            'CC': 'C',
                            'GG': 'G',
                            'TT': 'T'
                          }
    
    # keep only SNPs (non-length changing variants)
    SNP_sites = pd.read_csv(f"/n/scratch/users/s/sak0914/smk_output/{isolate}/lineage/SNP_sites.tsv.gz", sep='\t', compression='gzip').query("REF.str.len() == ALT.str.len()").drop_duplicates().reset_index(drop=True)
    
    # these are REF alleles
    SNP_sites.loc[SNP_sites['ALT']=='.', 'ALT'] = SNP_sites['REF']

    # Reference
    SNP_sites.loc[(SNP_sites['FILTER'].isin(['PASS', 'Amb'])) &
                  (SNP_sites['QUAL'] >= qual_thresh) &
                  (SNP_sites['DP'] >= min_read_depth) &
                  (SNP_sites['BQ'] >= BQ_thresh) &
                  (SNP_sites['MQ'] >= MQ_thresh) &
                  (SNP_sites['AF'] <= 0.25), 'ALLELE'] = SNP_sites['REF']
    
    # high quality variants
    SNP_sites.loc[(SNP_sites['FILTER']=='PASS') &
                  (SNP_sites['QUAL'] >= qual_thresh) &
                  (SNP_sites['DP'] >= min_read_depth) &
                  (SNP_sites['BQ'] >= BQ_thresh) &
                  (SNP_sites['MQ'] >= MQ_thresh) &
                  (SNP_sites['AF'] > 0.75), 'ALLELE'] = SNP_sites['ALT']
    
    # ambiguous variants -- use the IUPAC codes representing the two nucleotides (REF and ALT) at a given site
    # faster to not apply it to all rows, only on the ones you need
    SNP_sites.loc[(SNP_sites['FILTER']=='Amb') &
                  (SNP_sites['QUAL'] >= qual_thresh) &
                  (SNP_sites['DP'] >= min_read_depth) &
                  (SNP_sites['BQ'] >= BQ_thresh) &
                  (SNP_sites['MQ'] >= MQ_thresh) &
                  (SNP_sites['AF'] > 0.25), 'ALLELE'] = SNP_sites.loc[(SNP_sites['FILTER']=='Amb') &
                                                                      (SNP_sites['QUAL'] >= qual_thresh) &
                                                                      (SNP_sites['DP'] >= min_read_depth) &
                                                                      (SNP_sites['BQ'] >= BQ_thresh) &
                                                                      (SNP_sites['MQ'] >= MQ_thresh) &
                                                                      (SNP_sites['AF'] > 0.25)].apply(lambda row: iupac_two_nuc_codes.get(''.join(sorted(row['REF'] + row['ALT'])), np.nan), axis=1)
    
    # fill the rest with N = no information
    # this includes low coverage sites because we don't know what's happening there, so in the absence of other information, it could be anything
    # also includes areas with deletions -- those shouldn't be REF, technically they are '-', which is treated the same as N
    # gap characters ('-') and Ns are treated the same in iqtree
    SNP_sites['ALLELE'] = SNP_sites['ALLELE'].fillna('N')
    
    SNP_sites['ISOLATE'] = isolate

    # return a pivoted matrix
    return SNP_sites.pivot(index='ISOLATE', columns='POS', values='ALLELE')

In [165]:
SNP_matrix_full = []

for isolate in os.listdir("/n/scratch/users/s/sak0914/smk_output"):

    SNP_fName = f"/n/scratch/users/s/sak0914/smk_output/{isolate}/lineage/SNP_sites.tsv.gz"
    
    if os.path.isfile(SNP_fName):
        SNP_matrix = create_SNP_matrix_from_VCF(isolate)
        SNP_matrix_full.append(SNP_matrix)

SNP_matrix_full = pd.concat(SNP_matrix_full)

In [166]:
SNP_matrix_full

POS,34,35,36,37,38,39,40,41,42,43,...,4411523,4411524,4411525,4411526,4411527,4411528,4411529,4411530,4411531,4411532
ISOLATE,,,,,,,,,,,,,,,,,,,,,
00-R1210,G,T,G,T,G,G,A,A,C,G,...,A,G,A,T,A,C,G,T,C,G
SAMEA3392614,G,T,G,T,G,G,A,A,C,G,...,A,G,A,T,A,C,G,T,C,G
SAMEA8741574,G,T,G,T,G,G,A,A,C,G,...,A,G,A,T,A,C,G,T,C,G


In [169]:
# remove sites that are the same across all isolates because they have no signal
SNP_matrix_no_constant = SNP_matrix_full.loc[:, SNP_matrix_full.nunique(dropna=False) > 1]
SNP_matrix_no_constant.shape

(3, 56550)

In [170]:
SNP_matrix_no_constant

POS,1452,1849,4348,4349,4350,4351,4352,4353,4354,4355,...,4397736,4400246,4401732,4404346,4404960,4406116,4407200,4407588,4407927,4408923
ISOLATE,,,,,,,,,,,,,,,,,,,,,
00-R1210,T,C,N,N,N,N,N,N,N,N,...,C,A,C,T,C,C,G,T,T,C
SAMEA3392614,G,A,G,A,G,G,A,T,A,T,...,T,G,A,C,C,C,A,C,G,T
SAMEA8741574,G,C,G,A,G,G,A,T,A,T,...,C,G,C,C,G,A,A,T,T,C


In [ ]:
# remove sites where more than 10% of isolates have Ns -- this is a param that can be changed by the user

In [21]:
glob.glob("kraken_db/BAKTA/*")[0]

'kraken_db/BAKTA/01_R1134.HybridAsm.Bakta.Genome.fasta'

In [22]:
glob.glob("kraken_db/PGAP/*")[0]

'kraken_db/PGAP/01_R1134.HybridAsm.PGAP.Genome.fasta'

In [19]:
fName.replace('PGAP', 'BAKTA')

'kraken_db/BAKTA/01_R1134.HybridAsm.BAKTA.Genome.fasta'

In [14]:
# Example usage
compare_fasta_sequences("kraken_db/01_R1134.HybridAsm.Bakta.Genome.fasta", "kraken_db/01_R1134.HybridAsm.PGAP.Genome.fasta")

The sequences in the two FASTA files are the same.


In [2]:
todo_isolates = pd.read_csv("../data_cleaning/samples_for_F2.txt", header=None, sep='\t')
todo_isolates.shape

(4319, 1)

In [4]:
todo_isolates.loc[todo_isolates[0]=='SAMN37457963']

,0
3863,SAMN37457963


In [ ]:
python3 MtbQuantCNN/data_processing/variant_calling/F2_calculation.py data_cleaning/samples_for_F2.txt /n/data1/hms/dbmi/farhat/rollingDB/genomic_data